### Below, when I refer to "model level data" that's on the hybrid coordinates, whereas "pressure level data" is on pressure levels

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from math import nan
import xesmf as xe

from myPythonTools.CASutils import regrid_utils as regrid
from myPythonTools.CASutils import mapplot_utils as mymaps
import isla_interp_utils as isla_interp
import importlib
importlib.reload(regrid)
import warnings
warnings.filterwarnings('ignore')
from scipy import interpolate as intr
import dask

In [ ]:
# get the workers going
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '30GB',
    processes = 1,
    queue = 'casper',
    local_directory='$TMPDIR',
    resource_spec='select=1:ncpus=1:mem=30GB',
    project='P04010022',
    walltime='03:00:00',
    interface='mgt')

# scale up
cluster.scale(24)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

In [ ]:
client

In [ ]:
cluster

### Reading in a file on the f09 grid to get out the lons and lats for regridding

In [ ]:
ic = xr.open_dataset("/glade/campaign/cgd/cas/islas/python_savs/CESM3_dev/enso/ERA5_IC/fv0.9x1.25/julio/fv1x1/L58/ERA5_x_fv1x1_L58_rgC1_WO.1997-11-01-00000.nc").load()

### Reading in the ERA5 pressure level data and grabbing out 10 hPa and regridding horizontally to the f09 grid.  This will be our truth as it's the data on pressure levels provided by ERA5

In [ ]:
era5_plev = xr.open_dataset("/glade/campaign/collections/rda/data/ds633.0/e5.oper.an.pl/199711/e5.oper.an.pl.128_130_t.ll025sc.1997110100_1997110123.nc").isel(time=0)
era5_plev = era5_plev.sel(level=10., method='nearest') - 273.15

era5_plev_rg = regrid.regrid_conservative(era5_plev, era5_plev.longitude, era5_plev.latitude, ic.lon, ic.lat, reuse_wgts=False,
                                          wgtfile='/glade/derecho/scratch/islas/temp.nc')
era5_plev_rg = era5_plev_rg.load()

### Plotting up the ERA5 pressure level temperature field

In [ ]:
fig = plt.figure(figsize=(16,16))

ax = mymaps.contourmap_bothcontinents_robinson_pos(fig, era5_plev_rg.T, era5_plev_rg.lon, era5_plev_rg.lat,
                                                   2,-50,-10,'ERA5 pressure level data',0.05,0.35,0.8,0.97)

### Reading in the interpolated data from Jerry's code and then interpolating from hybrid to pressure using geocat vertical interpolation.  This started from the data on the 1280x640 grid

In [ ]:
oldic = xr.open_dataset("/glade/campaign/cesm/development/espwg/SMYLE-ERA5-L83/inputdata/cesm2_init/b.e21.SMYLE_ERA5_L83_IC.f09_g17.1997-11.01/1997-11-01/"+
                        "b.e21.SMYLE_ERA5_L83_IC.f09_g17.1997-11.01.cam.i.1997-11-01-00000.nc",
                       decode_times=False).load()
oldic['lon'] = ic.lon.values ; oldic['lat'] = ic.lat.values

In [ ]:
oldic_plev = isla_interp.interp_hybrid_to_pressure(
    oldic.T, oldic.PS, oldic.hyam, oldic.hybm, p0=1e5, new_levels=np.array([90000.,70000.,50000.,25000.,10000.,1000.]), method='log',
    lev_dim='lev', extrapolate=False, variable='temperature', t_bot=oldic.T.isel(lev=oldic.lev.size-1), phi_sfc=oldic.PHIS)

In [ ]:
oldic_plev = oldic_plev.T.sel(plev=100.*10.).isel(time=0)

In [ ]:
oldic_plev = oldic_plev.transpose('lat','lon')

In [ ]:
oldic_plev = oldic_plev - 273.15

### Reading in the interpolated data from Julio's code and then interpolating from hybrid to pressure using geocat vertical interpolation.  This also started from the data on the 1280x640 grid

In [ ]:
ic = xr.open_dataset("/glade/campaign/cgd/cas/islas/python_savs/CESM3_dev/enso/ERA5_IC/fv0.9x1.25/julio/fv1x1/L58/ERA5_x_fv1x1_L58_rgC1_WO.1997-11-01-00000.nc").load()

In [ ]:
ic_plev = isla_interp.interp_hybrid_to_pressure(
    ic.T, ic.PS, ic.hyam, ic.hybm, p0=1e5, new_levels=np.array([90000.,70000.,50000., 25000., 10000.,1000.]), method='log',
    lev_dim='lev', extrapolate=False, variable='temperature',t_bot = ic.T.isel(lev=ic.lev.size-1), phi_sfc=ic.PHIS)
ic_plev = ic_plev.load()

In [ ]:
ic_plev = ic_plev.T.sel(plev=100.*10.).isel(time=0)

In [ ]:
ic_plev = ic_plev.transpose('lat','lon')

In [ ]:
ic_plev = ic_plev - 273.15

### The difference between from the ERA5 pressure level baseline for Jerry's and Julio's script output

In [ ]:
fig = plt.figure(figsize=(16,16))

ax = mymaps.contourmap_bothcontinents_robinson_pos(fig, oldic_plev - era5_plev_rg.T, era5_plev_rg.lon, era5_plev_rg.lat,
                                                   0.1,-2,2,"Jerry's code",0.05,0.35,0.8,0.97)

ax = mymaps.contourmap_bothcontinents_robinson_pos(fig, ic_plev - era5_plev_rg.T, era5_plev_rg.lon, era5_plev_rg.lat,
                                                   0.1,-2,2,"Julio's code",0.38,0.68,0.8,0.97)

### The above shows that there are bigger differences in the 10 hPa values in Julio's code compared to Jerry's code when comparing to the ERA5 pressure level data

### Now I'm just trying to do the interpolation myself, starting from the same model level data as Julio's code i.e., the data at /glade/campaign/collections/rda/data/ds633.6/e5.oper.an.ml

### Reading in the ERA5 data

In [ ]:
era5 = xr.open_dataset(
    "/glade/campaign/collections/rda/data/ds633.6/e5.oper.an.ml/199711/e5.oper.an.ml.0_5_0_0_0_t.regn320sc.1997110100_1997110105.nc"
     ).isel(time=0)
era5_ps = xr.open_dataset(
    "/glade/campaign/collections/rda/data/ds633.6/e5.oper.an.ml/199711/e5.oper.an.ml.128_134_sp.regn320sc.1997110100_1997110105.nc"
     ).isel(time=0)
phisera5 = xr.open_dataset('/glade/u/home/islas/python/ERA5interp/phis/ERA5_phis.nc')
phisera5 = phisera5.rename({'g4_lon_3':'longitude', 'g4_lat_2':'latitude'})

### Regrid in the horizontal direction

In [ ]:
era5_rg = regrid.regrid_conservative(era5.T, era5.longitude, era5.latitude, ic.lon, ic.lat, reuse_wgts=False,
                                     wgtfile='/glade/derecho/scratch/islas/temp.nc')
era5_ps_rg = regrid.regrid_conservative(era5_ps.SP, era5.longitude, era5.latitude, ic.lon, ic.lat, reuse_wgts=True,
                                        wgtfile='/glade/derecho/scratch/islas/temp.nc')
phisera5_rg = regrid.regrid_conservative(phisera5.Z_GDS4_SFC, phisera5.longitude, phisera5.latitude,
                                       ic.lon, ic.lat, reuse_wgts=True,wgtfile='/glade/derecho/scratch/islas/temp.nc')

### Vertically interpolate using GeoCAT

In [ ]:
era5_plev_geocat = isla_interp.interp_hybrid_to_pressure(
    era5_rg, era5_ps_rg, era5.a_model, era5.b_model, p0=1, new_levels=np.array([90000.,70000.,50000., 25000., 10000.,1000.]), method='log',
    lev_dim='level', extrapolate=False, variable="temperature",
    t_bot = era5_rg.isel(level=era5_rg.level.size-1), phi_sfc = phisera5_rg)
era5_plev_geocat = era5_plev_geocat.load()

In [ ]:
era5_plev_geocat = era5_plev_geocat.sel(plev=10.*100.)

In [ ]:
era5_plev_geocat = era5_plev_geocat - 273.15

### Vertically interpolate using SciPy (like Julio)

In [ ]:
plevs = np.array([90000.,70000.,50000., 25000., 10000., 1000.])
pvals = era5.a_model + era5.b_model*era5_ps_rg
era5_plev_scipy = xr.DataArray(np.zeros([len(plevs),era5_rg.lat.size, era5_rg.lon.size]), dims=['plev','lat','lon'], coords=[plevs, era5_rg.lat, era5_rg.lon], name='era5_plev_scipy')
for ilon in np.arange(0,era5_rg.lon.size,1):
    print(ilon)
    for ilat in np.arange(0,era5_rg.lat.size,1):
        fintr = intr.interp1d(pvals.isel(lon=ilon, lat=ilat), era5_rg.isel(lon=ilon, lat=ilat), fill_value="extrapolate", kind = 'linear')
        era5_plev_scipy[:,ilat,ilon] = fintr(plevs)

In [ ]:
era5_plev_scipy = era5_plev_scipy.sel(plev=10.*100.)

In [ ]:
era5_plev_scipy = era5_plev_scipy - 273.15

In [ ]:
test = era5_plev_geocat - era5_plev_rg.T

In [ ]:
np.max(test)

### Plotting the difference between these interpolated fields and ERA5 pressure level data

In [ ]:
fig = plt.figure(figsize=(16,16))

test = era5_plev_geocat - era5_plev_rg.T
ax = mymaps.contourmap_bothcontinents_robinson_pos(fig, test, test.lon, test.lat,
                                                   0.1,-2,2,"GoeCat interpolation",0.05,0.35,0.8,0.97)

ax = mymaps.contourmap_bothcontinents_robinson_pos(fig, era5_plev_scipy - era5_plev_rg.T, era5_plev_rg.lon, era5_plev_rg.lat,
                                                   0.1,-2,2,"Scipy interpolation",0.38,0.68,0.8,0.97)

### The above shows that when I interpolate my way, conservative remapping in the horizontal using xESMF and then vertical interpolation either using GeoCAT or Scipy (as Julio) then I get even smaller errors than Jerry's code when comparing against the ERA5 pressure level data

### Now I'm testing the sensitivity to the ordering.  The above is interpolating horizontally and then vertically.  Now I'm interpolating vertically and then horizontally, using the GEOCAT vertical interpolation method

### Reading in the ERA5 data

In [ ]:
era5 = xr.open_dataset(
    "/glade/campaign/collections/rda/data/ds633.6/e5.oper.an.ml/199711/e5.oper.an.ml.0_5_0_0_0_t.regn320sc.1997110100_1997110105.nc"
     ).isel(time=0)
era5_ps = xr.open_dataset(
    "/glade/campaign/collections/rda/data/ds633.6/e5.oper.an.ml/199711/e5.oper.an.ml.128_134_sp.regn320sc.1997110100_1997110105.nc"
     ).isel(time=0)
phisera5 = xr.open_dataset('/glade/u/home/islas/python/ERA5interp/phis/ERA5_phis.nc')
phisera5 = phisera5.rename({'g4_lon_3':'longitude', 'g4_lat_2':'latitude'})

### Vertical interpolation

In [ ]:
era5_plev = isla_interp.interp_hybrid_to_pressure(
    era5.T, era5_ps.SP, era5.a_model, era5.b_model, p0=1, new_levels=np.array([90000.,70000.,50000., 25000., 10000.,1000.]), method='log',
    lev_dim='level', extrapolate=False, variable="temperature",
    t_bot = era5.T.isel(level=era5.level.size-1), phi_sfc = phisera5)

### Horizontal interpolation (conservative)

In [ ]:
era5_plev_rg_cons = regrid.regrid_conservative(era5_plev, era5_plev.longitude, era5_plev.latitude, ic.lon, ic.lat, reuse_wgts=False, 
                                               wgtfile='/glade/derecho/scratch/islas/temp.nc')

### Horizontal interpolation (bilinear)

In [ ]:
#era5_plev = era5_plev.rename({'longitude':'lon', 'latitude':'lat'})
regridder = xe.Regridder(era5_plev, ic, 'bilinear', reuse_weights=False, filename='/glade/derecho/scratch/islas/temp.nc')
era5_plev_rg_bilin = regridder(era5_plev)

In [ ]:
era5_plev_rg_cons = era5_plev_rg_cons - 273.15
era5_plev_rg_bilin = era5_plev_rg_bilin - 273.15

In [ ]:
fig = plt.figure(figsize=(16,16))

ax = mymaps.contourmap_bothcontinents_robinson_pos(fig, era5_plev_rg_cons.sel(plev=10*100.) - era5_plev_rg.T, era5_plev_rg_cons.lon, era5_plev_rg_cons.lat,
                                                   0.1,-2,2,"Conservative horizontal",0.05,0.35,0.8,0.97)

ax = mymaps.contourmap_bothcontinents_robinson_pos(fig, era5_plev_rg_bilin.sel(plev=10.*100.) - era5_plev_rg.T, era5_plev_rg_bilin.lon, era5_plev_rg_bilin.lat,
                                                   0.1,-2,2,"Bilinear horizontal",0.38,0.68,0.8,0.97)

### Whatever I do, whether that be doing pressure level interpolation first, doing horizontal interpolation first, using geocat or scipy for vertical interpolation or using conservative of bilinear in terpolation for the horizontal regridding, I can't reproduce as large differences as we see with Julio's script